In [1]:
import sys 
sys.path.append('..')
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:

from src.datasets.mimic_cxr_dataset import MIMICCXRDataModule
from pytorch_lightning import Trainer, seed_everything
import torchvision.transforms as T
import torch
# sets seeds for numpy, torch, python.random and PYTHONHASHSEED.
seed_everything(42)


/home/mlmi-matthias/anaconda3/envs/mlmi_caghan/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Global seed set to 42


42

In [6]:
from src.datasets.mimic_cxr_rachet import MIMICCXRRachetDataModule

In [7]:
rachet_hyperparameters = {
    
    
}
MIMICCXRRachetDataModule(abs)

src.datasets.mimic_cxr_rachet.MIMICCXRRachetDataModule

In [ ]:
import pandas as pd

In [ ]:
validate_indices = pd.read_csv('MIMIC_AP_PA_validate.csv')

In [ ]:
validate_indices

In [ ]:
len(validate_indices)

In [ ]:
validate_indices.iloc[0]["img_path"]

In [ ]:
validate_indices.iloc[0]["report"]

In [ ]:
augmentations = {'train':
        T.Compose(
        [
        # T.Resize((224, 224)),
        # T.ToTensor(),
        
            T.RandomRotation(10),
            T.ToTensor(),
            #T.Normalize(mean=(0.4719, 0.4719, 0.4719), std=(0.3029, 0.3029, 0.3029))
        ]),
        'val':
        T.Compose(
        [
            #T.Resize((224, 224)),
            #T.ToTensor(),
            #T.RandomRotation(10),
            T.ToTensor(),
            #T.Normalize(mean=(0.4719, 0.4719, 0.4719), std=(0.3029, 0.3029, 0.3029))
        ]),
        'test':
        T.Compose(
        [
            #T.Resize((224, 224)),
            #T.ToTensor(),
            T.RandomRotation(10),
            T.ToTensor(),
            #T.Normalize(mean=(0.4719, 0.4719, 0.4719), std=(0.3029, 0.3029, 0.3029))
        ])
    }


# Hyperparameters
NUM_DATA_WORKERS  = 4
ONLY_IMAGES = False
BATCH_SIZE = 1
NUM_EPOCHS = 200
LIMIT_NUM_SAMPLES = 5000

if os.getcwd().startswith('/home/mlmi-matthias'):
    ACCELERATOR = "gpu"
    DEVICES = [4,5,6,7]
    PRETRAINED_CLIP_PATH = '/home/mlmi-matthias/Caghan/pretrained_models/PubMedCLIP_ViT32.pth'
    PRETRAINED_GPT2_PATH = "/home/mlmi-matthias/Caghan/pretrained_models/gpt2-pytorch_model.bin"
    MIMIC_CXR_DCM_PATH = '/home/mlmi-matthias/physionet.org/files/mimic-cxr/2.0.0/files/'
    MIMIC_CXR_JPG_PATH = "/home/mlmi-matthias/physionet.org/files/mimic-cxr-jpg/2.0.0/files/"
    SPLIT_PATH = '/home/mlmi-matthias/Caghan/mlmi-vqa/data/external/'

elif os.getcwd().startswith('/Users/caghankoksal'):
    PRETRAINED_CLIP_PATH = '/Users/caghankoksal/Desktop/development/PubMedCLIP_ViT32.pth'
    PRETRAINED_GPT2_PATH = "/Users/caghankoksal/Desktop/development/TransformerPlay/gpt2-pytorch_model.bin"
    ACCELERATOR = "cpu"
    DEVICES = 1
    MIMIC_CXR_DCM_PATH = '/Users/caghankoksal/Desktop/development/Flamingo-playground/physionet.org/files/mimic-cxr/2.0.0/files/'
    MIMIC_CXR_JPG_PATH = '/Users/caghankoksal/Desktop/development/physionet.org/files/mimic-cxr-jpg/2.0.0/files/'
    SPLIT_PATH = '/Users/caghankoksal/Desktop/SS2022/mlmi-vqa/data/external/'


IMAGE_TYPE = "jpg"
TOKENIZER  = "gpt2"
PREPROCESSED = True

mimic_datamodule = MIMICCXRDataModule(MIMIC_CXR_DCM_PATH, MIMIC_CXR_JPG_PATH, 
                                    transforms=augmentations, only_images=False, batch_size=BATCH_SIZE,
                                    limit_num_samples=LIMIT_NUM_SAMPLES, num_data_workers=NUM_DATA_WORKERS,
                                    tokenizer="gpt2",image_type="jpg", split_path=SPLIT_PATH, preprocessed=PREPROCESSED
)


In [ ]:

train_loader = mimic_datamodule.train_dataloader()
val_loader = mimic_datamodule.val_dataloader()

In [ ]:
mimic_datamodule.train_dataset[0]

In [ ]:
len(mimic_datamodule.train_dataset)

In [ ]:
len(mimic_datamodule.validation_dataset)

In [ ]:
len(mimic_datamodule.test_dataset)

In [ ]:
len(train_loader)

In [ ]:
for batch in train_loader:
    print("Image shape : ",batch["image"].shape)
    print(batch["text"][0]) # First sample in the batch
    print("Input Ids shape : ",batch["input_ids"].shape)
    print("Targets shape : ",batch["targets"].shape)
    break

In [ ]:
from src.models.multimodal.flamingo_module import FlamingoModule
import pytorch_lightning as pl

In [ ]:
VOCAB_SIZE_OF_TOKENIZER = 50257 # mimic_datamodule.train_dataset.tokenizer.vocab_size
LANGUAGE_MODEL = 'gpt2'
NUM_TOKENS = VOCAB_SIZE_OF_TOKENIZER +3 if LANGUAGE_MODEL=="gpt2" else 31092
FLAMINGO_EMBED_DIM = 768
DEPTH = 12
NUM_HEADS = 8
ATT_HEAD_DIM = 64
CROOS_ATT_EVERY=3
MEDIA_TOKEN_ID = mimic_datamodule.train_dataset.tokenizer.all_special_ids[mimic_datamodule.train_dataset.tokenizer.all_special_tokens.index('<image>')]
PERCEIVER_NUM_LATENTS = 64
PERCEIVER_DEPTH = 2
IMAGE_ENCODER = "clip"


print(  "VOCAB_SIZE_OF_TOKENIZER : ",VOCAB_SIZE_OF_TOKENIZER, "\n"
        "LANGUAGE_MODEL : ",LANGUAGE_MODEL, "\n"
        "NUM_TOKENS : ",NUM_TOKENS, "\n"
        "FLAMINGO_EMBED_DIM : ",FLAMINGO_EMBED_DIM, "\n"
        "DEPTH : ",DEPTH, "\n"
        "NUM_HEADS : ",NUM_HEADS, "\n"
        "ATT_HEAD_DIM : ",ATT_HEAD_DIM, "\n"
        "CROOS_ATT_EVERY : ",CROOS_ATT_EVERY, "\n"
        "MEDIA_TOKEN_ID : ",MEDIA_TOKEN_ID, "\n"
        "PERCEIVER_NUM_LATENTS : ",PERCEIVER_NUM_LATENTS, "\n"
        "PERCEIVER_DEPTH : ",PERCEIVER_DEPTH, "\n"
        "IMAGE_ENCODER : ",IMAGE_ENCODER, "\n"
        "PRETRAINED_CLIP_PATH : ",PRETRAINED_CLIP_PATH, "\n"
        "PRETRAINED_GPT2_PATH : ",PRETRAINED_GPT2_PATH, "\n")

In [ ]:
hyperparams = {
    'pretrained_clip_path': PRETRAINED_CLIP_PATH,
    'warmup_steps': 569,
    'num_tokens': NUM_TOKENS,
    'dim': FLAMINGO_EMBED_DIM,
    'depth': DEPTH,
    'num_heads': NUM_HEADS,
    'dim_head': ATT_HEAD_DIM,
    'cross_attn_every': CROOS_ATT_EVERY,
    'media_token_id': MEDIA_TOKEN_ID,
    'perceiver_num_latents': PERCEIVER_NUM_LATENTS,
    'perceiver_depth': PERCEIVER_DEPTH,
    'image_encoder': IMAGE_ENCODER,
    'language_model': LANGUAGE_MODEL,
    'pretrained_gpt2_path': PRETRAINED_GPT2_PATH,
    "classification_mode": False,
    "use_positional_embedding":False
}

In [ ]:
from src.models.multimodal.flamingo_module import FlamingoModule

In [ ]:
model = FlamingoModule(**hyperparams)

In [ ]:
iterator = iter(val_loader)

In [ ]:
batch = next(iterator)
out, attns = model(batch, return_attn = True)


In [ ]:
model(batch)


In [ ]:
import numpy as np
import cv2
from torchvision.transforms import transforms
import matplotlib.pyplot as plt

In [ ]:
# Attention Maps
batch = next(iterator)
out, attns = model(batch, return_attn = True)
img = batch["image"]
# Stack attention maps
att_mat = torch.stack(attns)
att_mat = att_mat.squeeze(1)
# Average the attention weights across all heads.
#att_mat = torch.mean(att_mat, dim=0)

# Take the Last attention map
att_mat = att_mat[-1,:,:]
#Use Classification Token's attention
cur_attn_map = att_mat[0,1:]

grid_size = int(np.sqrt(cur_attn_map.shape[0])) # 49, 49
mask = cur_attn_map.reshape(grid_size, grid_size).detach().numpy()

im = transforms.ToPILImage()(img.squeeze(0))
# Maybe use interpolate instead of resize
mask2 = cv2.resize(mask / mask.max(), im.size)[..., np.newaxis]
result = (torch.tensor(mask2[:,:,0]) * img[0][0])
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(16, 16))

ax1.set_title('Original')
ax2.set_title('Attention Map')
_ = ax1.imshow(im)
_ = ax2.imshow(result)

In [ ]:
batch = next(iterator)
out, attns = model(batch, return_attn = True)
# Attention Maps
img = batch["image"]
# Stack attention maps


In [ ]:
from src.preprocess.visualize_attention import plot_attention_maps

In [ ]:
import numpy as np
import cv2
from torchvision.transforms import transforms
import matplotlib.pyplot as plt
import torch
from torch import nn

def plot_attention_maps2(attns, img, num_heads=12):

    
    fig, ax = plt.subplots(num_heads, 4)
    fig.set_size_inches(18.5, 70)


    # Stack attention maps
    att_maps = torch.stack(attns)
    att_maps = att_maps.squeeze(1)
    # Average the attention weights across all heads.
    #att_mat = torch.mean(att_mat, dim=0)


    for j in range(num_heads):
        #fname = os.path.join(args.output_dir, "attn-head" + str(j) + ".png")
        #plt.imsave(fname=fname, arr=attentions[j], format='png')
        #print(f"{fname} saved.")

        # Take the Last attention map
        cur_attention_head = att_maps[j,:,:]
        #Use Classification Token's attention
        cur_attn_map = cur_attention_head[0,1:]

        grid_size = int(np.sqrt(cur_attn_map.shape[0])) # 49, 49
        mask = cur_attn_map.reshape(grid_size, grid_size).detach().numpy()

        im = transforms.ToPILImage()(img.squeeze(0))
        # Maybe use interpolate instead of resize
        
        interpol_mask = nn.functional.interpolate(torch.tensor(mask).unsqueeze(0).unsqueeze(1), scale_factor=32, mode="nearest")
        interpol_mask = interpol_mask[0,0,:,:].unsqueeze(2)
        result_interpol = (torch.tensor(interpol_mask[:,:,0]) * img[0][0])
        
        resize_mask = cv2.resize(mask / mask.max(), im.size)[..., np.newaxis]
        result_resize = (torch.tensor(resize_mask[:,:,0]) * img[0][0])


        ax[j][0].imshow(im)
        ax[j][0].set_title("Orginal")
        
        
        ax[j][1].imshow(im)
        #ax[j][1].imshow(mask, cmap='hot', alpha=0.7)
        ax[j][1].imshow(resize_mask, cmap='hot', alpha=0.7)
        ax[j][1].set_title("Head Attention" + str(j))
        
        ax[j][2].imshow(result_resize)
        ax[j][2].set_title("Head result_resize Attention" + str(j))

        ax[j][3].imshow(resize_mask)
        ax[j][3].set_title("Head resize_mask Attention" + str(j))

    
    plt.show()

In [ ]:
plot_attention_maps2(attns, img, num_heads=12)

In [ ]:
batch = next(iterator)
out, attns = model(batch, return_attn = True)
# Attention Maps
img = batch["image"]
# Stack attention maps


In [ ]:
# To account for residual connections, we add an identity matrix to the
# attention matrix and re-normalize the weights.
residual_att = torch.eye(att_mat.size(1))
aug_att_mat = att_mat + residual_att
aug_att_mat = aug_att_mat / aug_att_mat.sum(dim=-1).unsqueeze(-1)

# Recursively multiply the weight matrices
joint_attentions = torch.zeros(aug_att_mat.size())
joint_attentions[0] = aug_att_mat[0]
im = transforms.ToPILImage()(img.squeeze(0))
for n in range(1, aug_att_mat.size(0)):
    joint_attentions[n] = torch.matmul(aug_att_mat[n], joint_attentions[n-1])
print("JA shape: ",joint_attentions.shape)
# Attention from the output token to the input space.

grid_size = int(np.sqrt(aug_att_mat.size(-1)))
mask = joint_attentions[0, 1:].reshape(grid_size, grid_size).detach().numpy()
mask = cv2.resize(mask / mask.max(), im.size)[..., np.newaxis]
result = (mask * im).astype("uint8")

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(16, 16))

ax1.set_title('Original')
ax2.set_title('Attention Map')
_ = ax1.imshow(im)
_ = ax2.imshow(result)



COMET_API_KEY = "F2L19mQwKXSoeF1IYEDA2AeHD",
PROJECT_KEY = "flamingo-playground",
import os
from pytorch_lightning.loggers import CometLogger

comet_logger = CometLogger(
    api_key= "F2L19mQwKXSoeF1IYEDA2AeHD",
    project_name="flamingo-gpt2")


In [ ]:
#from pytorch_lightning.callbacks import LearningRateMonitor
#lr_monitor = LearningRateMonitor(logging_interval='step')

trainer = pl.Trainer(max_epochs=6,
                     accelerator="cpu", devices=1,
                     callbacks=[lr_monitor],
                     log_every_n_steps=1,
                      )



In [ ]:
#trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)

In [ ]:
#!tensorboard --logdir=lightning_logs

In [ ]:
#from pytorch_lightning.utilities.model_summary import LayerSummary

In [ ]:
#summary = LayerSummary(model.flamingo_palm.token_emb)
#summary.num_parameters

In [ ]:
# from transformers import AutoTokenizer


In [ ]:
# INFERENCE

In [ ]:
import torch
import torchvision
import torchvision.transforms as T
from PIL import Image
from transformers import GPT2Tokenizer
import numpy as np

from torch import nn as nn
import torch.nn.functional as F
def generate(image, context, ntok=20):
    for _ in range(ntok):
        out = model({'image': image,'input_ids': context })
        logits = out[:, -1, :]
        indices_to_remove = logits < torch.topk(logits, 10)[0][..., -1, None]
        logits[indices_to_remove] = np.NINF
        next_tok = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1).squeeze(1)
        # EOC
        if next_tok ==  50259:
            break
        context = torch.cat([context, next_tok.unsqueeze(-1)], dim=-1)
    return context


tokenizer = mimic_datamodule.train_dataset.tokenizer

In [ ]:
START_FROM_CHECKPOINT = True

if START_FROM_CHECKPOINT:
    # MIMIC CXR trained without positional embedding
    CHECKPOINT_PATH = '/home/mlmi-matthias/Caghan/mlmi-vqa/notebooks/lightning_logs/version_20/checkpoints/epoch=114-val_loss=0.84-other_metric=0.00.ckpt'
    print("Pretrained Flamingo Model is loaded from checkpoint : ",CHECKPOINT_PATH)
    if os.getcwd().startswith('/home/mlmi-matthias'):
        model.load_state_dict(torch.load(CHECKPOINT_PATH)["state_dict"],strict=False)
    else:
        ckpt_folder_path = "/Users/caghankoksal/Desktop/logs_from_cluster/lightning_logs_flamingo-gpt2/"
        os.listdir(ckpt_folder_path)
        model_folder_name = '9_da477150ff4a4f50866683712eb8c707'
        os.listdir(os.path.join(ckpt_folder_path,model_folder_name))
        os.listdir(os.path.join(ckpt_folder_path,model_folder_name, "checkpoints"))
        CHECKPOINT_PATH = os.path.join(ckpt_folder_path,model_folder_name,"checkpoints",'epoch=176-step=49206.ckpt')

In [ ]:
model.eval()

In [ ]:
load_val = iter(val_loader)


In [ ]:
batch = next(load_val)
print(batch["text"]),
img = T.ToPILImage()(batch["image"][0])
img

In [ ]:
context   = torch.tensor([tokenizer.encode('<|endoftext|> <image> Output:')])
out = generate(batch["image"], context, ntok=180)
tokenizer.decode(out[0])

In [ ]:
context   = torch.tensor([tokenizer.encode('<|endoftext|> <image> Output: ')])
out = generate(batch["image"], context, ntok=180)
tokenizer.decode(out[0])

In [ ]:
context   = torch.tensor([tokenizer.encode('<|endoftext|> <image> Output: FINDINGS: ')])
out = generate(batch["image"], context, ntok=180)
tokenizer.decode(out[0])

In [ ]:
context   = torch.tensor([tokenizer.encode('<|endoftext|> <image> Question: Is there a penumothorax? If yes, explain why? Answer: ')])
out = generate(batch["image"], context, ntok=180)
tokenizer.decode(out[0])

In [ ]:
context   = torch.tensor([tokenizer.encode('<|endoftext|> <image> Output: ')])
out = generate(batch["image"], context, ntok=180)
tokenizer.decode(out[0])

In [ ]:
batch = next(load_val)
print(batch["text"]),
img = T.ToPILImage()(batch["image"][0])
img

In [ ]:
context   = torch.tensor([tokenizer.encode('<|endoftext|> <image> Output: ')])
out = generate(batch["image"], context, ntok=180)
tokenizer.decode(out[0])

In [ ]:
gts_report = batch["text"][0].split('<|endoftext|> <image> Output:')[1].split('<EOC>')[0].rstrip().strip()

In [ ]:
res_model = tokenizer.decode(out[0]).split('<|endoftext|> <image> Output:')[1].split('<EOC>')[0].rstrip().strip()

In [ ]:
gts ={'img_id': gts_report}
res = {'img_id': res_model}

In [ ]:
# TEXT EVALUATION 

In [ ]:
#!pip install pycocoevalcap

In [ ]:
val_loader = mimic_datamodule.val_dataloader()

In [ ]:
from tqdm import tqdm as tqdm

In [ ]:
gts = {}
preds = {}
for i,batch in tqdm(enumerate(val_loader)):
    context   = torch.tensor([tokenizer.encode('<|endoftext|> <image> Output: ')])
    out = generate(batch["image"], context, ntok=180)
    models_output = tokenizer.decode(out[0])

    preds[i] = [models_output.split('<|endoftext|> <image> Output:')[1].split('<EOC>')[0].rstrip().strip()]
    
    gts[i] = [batch["text"][0].split('<|endoftext|> <image> Output:')[1].split('<EOC>')[0].rstrip().strip()]


gts

In [ ]:
np.save('preds_mimic_cxr_5000samples', preds)

In [ ]:
np.save('gts_mimic_cxr_5000samples.npy', gts)

In [ ]:
from pycocoevalcap.bleu.bleu import Bleu
from pycocoevalcap.meteor.meteor import Meteor
from pycocoevalcap.rouge.rouge import Rouge
from pycocoevalcap.cider.cider import Cider
from pycocoevalcap.spice.spice import Spice

scorers = [
    (Bleu(4), ["Bleu_1", "Bleu_2", "Bleu_3", "Bleu_4"]),
    #(Meteor(),"METEOR"),
    (Rouge(), "ROUGE_L"),
    (Cider(), "CIDEr"),
    #(Spice(), "SPICE")
        ]

# =================================================
# Compute scores
# =================================================
for scorer, method in scorers:
    print('computing %s score...'%(scorer.method()))
    score, scores = scorer.compute_score(gts, preds)
    if type(method) == list:
        for sc, scs, m in zip(score, scores, method):
            #self.setEval(sc, m)
            #self.setImgToEvalImgs(scs, gts.keys(), m)
            print("%s: %0.3f"%(m, sc))
    else:
        #self.setEval(score, method)
        #self.setImgToEvalImgs(scores, gts.keys(), method)
        print("%s: %0.3f"%(method, score))
#self.setEvalImgs()

In [ ]:
class MIMICEvalCap:
    def __init__(self, true_gt, preds):

        self.preds = preds
        self.true_gts = true_gt

        self.eval = dict()
        self.imgToEval = dict()

    def preprocess(self, s):
        s = s.replace('\n', '')
        s = s.replace('<s>', '')
        s = s.replace('</s>', '')
        # s = s.translate(str.maketrans('', '', '0123456789'))
        # s = s.translate(str.maketrans('', '', string.punctuation))
        return s

    def evaluate(self):

        gts = dict()
        res = dict()

        # Sanity Checks
        assert len(self.preds.keys()) == len(self.true_gts.keys())

        # =================================================
        # Pre-process sentences
        # =================================================
        #print('tokenization...')
        #for i in range(self.pred_df.shape[0]):
        #    pred_text = ' '.join(word_tokenize(self.preprocess(self.pred_df[i][0])))
        #    true_text = ' '.join(word_tokenize(self.preprocess(self.true_df[i][0])))

        #    res[i] = [pred_text]
        #    gts[i] = [true_text]
        
        res = self.preds
        gts = self.true_gts

        # =================================================
        # Set up scorers
        # =================================================
        print('setting up scorers...')
        scorers = [
            (Bleu(4), ["Bleu_1", "Bleu_2", "Bleu_3", "Bleu_4"]),
            (Meteor(),"METEOR"),
            (Rouge(), "ROUGE_L"),
            (Cider(), "CIDEr"),
            (Spice(), "SPICE")
        ]

        # =================================================
        # Compute scores
        # =================================================
        for scorer, method in scorers:
            print('computing %s score...'%(scorer.method()))
            score, scores = scorer.compute_score(gts, res)
            if type(method) == list:
                for sc, scs, m in zip(score, scores, method):
                    self.setEval(sc, m)
                    self.setImgToEvalImgs(scs, gts.keys(), m)
                    print("%s: %0.3f"%(m, sc))
            else:
                self.setEval(score, method)
                self.setImgToEvalImgs(scores, gts.keys(), method)
                print("%s: %0.3f"%(method, score))
        self.setEvalImgs()

    def setEval(self, score, method):
        self.eval[method] = score

    def setImgToEvalImgs(self, scores, imgIds, method):
        for imgId, score in zip(imgIds, scores):
            if not imgId in self.imgToEval:
                self.imgToEval[imgId] = dict()
                self.imgToEval[imgId]["image_id"] = imgId
            self.imgToEval[imgId][method] = score

    def setEvalImgs(self):
        self.evalImgs = [eval for imgId, eval in self.imgToEval.items()]

In [ ]:
MIMICEvalCap(gts, preds).evaluate()

In [ ]:
# BEAM SEARCH

In [ ]:
import operator
import torch
import torch.nn as nn
import torch.nn.functional as F
from queue import PriorityQueue
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

SOS_token = 0
EOS_token = 1
MAX_LENGTH = 50


In [ ]:
out.shape

In [ ]:
class BeamSearchNode(object):
    def __init__(self, hiddenstate, previousNode, wordId, logProb, length):
        '''
        :param hiddenstate:
        :param previousNode:
        :param wordId:
        :param logProb:
        :param length:
        '''
        self.h = hiddenstate
        self.prevNode = previousNode
        self.wordid = wordId
        self.logp = logProb
        self.leng = length

    def eval(self, alpha=1.0):
        reward = 0
        # Add here a function for shaping a reward

        return self.logp / float(self.leng - 1 + 1e-6) + alpha * reward

In [ ]:
def beam_decode(target_tensor, decoder_hiddens, encoder_outputs=None):
    '''
    :param target_tensor: target indexes tensor of shape [B, T] where B is the batch size and T is the maximum length of the output sentence
    :param decoder_hidden: input tensor of shape [1, B, H] for start of the decoding
    :param encoder_outputs: if you are using attention mechanism you can pass encoder outputs, [T, B, H] where T is the maximum length of input sentence
    :return: decoded_batch
    '''

    beam_width = 10
    topk = 1  # how many sentence do you want to generate
    decoded_batch = []

    # decoding goes sentence by sentence
    for idx in range(target_tensor.size(0)):
    
        decoder_hidden = decoder_hiddens[:, idx, :].unsqueeze(0)
        encoder_output = encoder_outputs[:,idx, :].unsqueeze(1)

        # Start with the start of the sentence token
        decoder_input = torch.LongTensor([[SOS_token]], device=device)

        # Number of sentence to generate
        endnodes = []
        number_required = min((topk + 1), topk - len(endnodes))

        # starting node -  hidden vector, previous node, word id, logp, length
        node = BeamSearchNode(decoder_hidden, None, decoder_input, 0, 1)
        nodes = PriorityQueue()

        # start the queue
        nodes.put((-node.eval(), node))
        qsize = 1

        # start beam search
        while True:
            # give up when decoding takes too long
            if qsize > 2000: break

            # fetch the best node
            score, n = nodes.get()
            decoder_input = n.wordid
            decoder_hidden = n.h

            if n.wordid.item() == EOS_token and n.prevNode != None:
                endnodes.append((score, n))
                # if we reached maximum # of sentences required
                if len(endnodes) >= number_required:
                    break
                else:
                    continue

            # decode for one step using decoder
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, encoder_output)

            # PUT HERE REAL BEAM SEARCH OF TOP
            log_prob, indexes = torch.topk(decoder_output, beam_width)
            nextnodes = []

            for new_k in range(beam_width):
                decoded_t = indexes[0][new_k].view(1, -1)
                log_p = log_prob[0][new_k].item()

                node = BeamSearchNode(decoder_hidden, n, decoded_t, n.logp + log_p, n.leng + 1)
                score = -node.eval()
                nextnodes.append((score, node))

            # put them into queue
            for i in range(len(nextnodes)):
                score, nn = nextnodes[i]
                nodes.put((score, nn))
                # increase qsize
            qsize += len(nextnodes) - 1

        # choose nbest paths, back trace them
        if len(endnodes) == 0:
            endnodes = [nodes.get() for _ in range(topk)]

        utterances = []
        for score, n in sorted(endnodes, key=operator.itemgetter(0)):
            utterance = []
            utterance.append(n.wordid)
            # back trace
            while n.prevNode != None:
                n = n.prevNode
                utterance.append(n.wordid)

            utterance = utterance[::-1]
            utterances.append(utterance)

        decoded_batch.append(utterances)

    return decoded_batch

In [ ]:
# Generate Attention Map and Result

In [ ]:
load_val = iter(val_loader)
batch = next(load_val)
out, attns = model(batch, return_attn = True)
# Attention Maps
img = batch["image"]
# Stack attention maps


In [ ]:
batch["text"]

In [ ]:
context   = torch.tensor([tokenizer.encode('<|endoftext|> <image> Output: ')])
out = generate(batch["image"], context, ntok=180)
tokenizer.decode(out[0])

In [ ]:
plot_attention_maps2(attns, img, num_heads=12)